In [0]:
import requests
import re
import xlwt
import json



wb = xlwt.Workbook()
ws = wb.add_sheet('Data')




n = 0

tilte = ['Producturl','Productid','Producttitle','SizeType','Size','Color','StockStatus','Orinalprice','Currentprice','Offerprice','imgeurl','Descrption']

for index,head in enumerate(tilte):
	print(head)
	ws.write(0, index, head)
 

url_list = ['https://www.landsend.com/products/womens-plus-size-active-yoga-pants/id_338352?attributes=13774,43307,44139,44256',
'https://www.landsend.com/products/womens-relaxed-supima-cotton-long-sleeve-crewneck-t-shirt/id_302536?attributes=44967,4408&dysku=4766502'
'https://www.landsend.com/products/mens-short-sleeve-comfort-first-mesh-polo-shirt/id_331411?attributes=11657,43307,43322,44255,44896,44967,45022',
'https://www.landsend.com/products/womens-v-neck-cross-back-bikini-top-swimsuit-adjustable-straps-print/id_342571?attributes=8528,43307,43325,43393,44256,44405',
'https://www.landsend.com/products/womens-expedition-fleece-quarter-zip-long-underwear/id_344708?attributes=2685,43307,43328,44256',
'https://www.landsend.com/products/girls-hoodie-pullover-sweatshirt/id_210527?attributes=3153,44257,44371,44827',
'https://www.landsend.com/products/mens-board-short-swim-trunks/id_332276?attributes=11657,43307,43325,43394,44255',
'https://www.landsend.com/products/mens-super-t/id_333358?attributes=18890,43307,43322,43381,44068,44255,44896,44967',
'https://www.landsend.com/products/mens-big-tall-long-sleeve-super-t/id_333424?attributes=21938,43307,43322,43381,44068,44255',
'https://www.landsend.com/products/mens-lightweight-comfort-boat-shoes/id_340553?attributes=12323,43308,43330,44255,45441',
'https://www.landsend.com/products/mens-full-length-turkish-terry-robe/id_151816?attributes=32671,43307,43327,44255',
'https://www.landsend.com/products/womens-long-sleeve-polo-rugby-shirt-stripe/id_339213?attributes=4735,43307,43322,44256,44967,45022',
'https://www.landsend.com/products/womens-active-long-sleeve-t-shirt/id_336482?attributes=13330,43307,44139,44256,44967']

for inx,url in enumerate(url_list):
	print(inx)
	print(url)
	res = requests.get(url)
	# print('res:',res)

	Content = res.content.decode('UTF-8')
	with open(str()+ 'homepage.html','w',encoding = 'UTF-8') as d:
		d.write(Content)

	productTitle = re.findall('itemprop\=\"name\"\>([^>]*?)</h1>',Content)
	print('productTitle:',productTitle[0])

	productid = re.findall('<link\s*rel\=\"canonical\"[^>]*?id\_([^>]*?)\">',Content)
	# print('productid:',productid[0])
	
	
	# price_block = re.findall('(<product-price[\w\W]*?</product-price>)',Content)
	
	# price = re.findall('')
	

	
	
	jsonContent = re.findall('\"application\/json\"\>([\w\W]*?\}})\<',Content)


	data = re.sub('&q;','"',jsonContent[0])
	# print('data::',data)
	with open('jsondata.txt','w',encoding =  'UTF-8') as r:
		r.write(data)

	jsondata = json.loads(data)
	# print('jsondata::',jsondata)
	
	
	des = jsondata['/api/v2/product/'+str(productid[0])]['productDetail']['productCopies'][0]['featureBullets']
	# print('descrption:',des)

	skus_s = jsondata['/api/v2/product/'+str(productid[0])]['productDetail']['skus']
	for skus in skus_s:
		color = skus['color']['values'][0]['label']
		# print('color:',color)
		
		ColorCode = skus['colorCode']
		print('ColorCode:',ColorCode)
		
		styeNumber = skus['styleNumber']
		print('styeNumber:-->',styeNumber)

		sizetype = skus['attributeTypes']
		# print('sizetype:',sizetype)
		if sizetype:
			size_type = sizetype[0]['values'][0]['label']
			# print('sizetype:',size_type)
		else:
			size_type = ''
	
		size = skus['size']['values'][0]['label']
		# print('size:',size)
	
		originalPrice = skus['price']['originalPrice']
		# print('originalPrice:',originalPrice)
		
		currentprice = skus['price']['currentPrice']
		# print('currentprice:',currentprice)
		
		stockStatus =skus['lowQuantity']
		# print('stockStatus:',stockStatus)
		if stockStatus == False:
			stockdteail = 'INSTOCK'
			# print('StockStatus::',stockdteail)
		else:
			stockdteail = 'OUTOFSTOCK'
			# print('StockStatus:',stockdteail)
	
		
		
		images = skus['images']
		images_list = []
		for image in images:
			# print('image::',image['imageUrl'])
			# input('----------')
			images_list.append(image['imageUrl'])
		final_images = ', '.join(images_list)
		# print('final_images::',final_images)
		
		
		
		offerurl = 'https://pc-api.landsend.com/prod/0.0.2/SPLASH/inetpage?ids=' + str(productid[0])
		
		res1 = requests.get(offerurl)
		offerContent = res1.content.decode('UTF-8')
		with open('offer.json','w',encoding = 'UTF-8') as r:
			r.write(offerContent)
		
		offerdata = json.loads(offerContent)
		productid_offerdata = productid[0]
		offerdata = offerdata[productid_offerdata] 
		
		styles = offerdata['styles']
		
		styeNumber = str(styeNumber)
		offerPrice = offerdata['styles'][styeNumber]['color_size'][ColorCode]
		
		print('originalPrice:',originalPrice)
		print('color:',color)
		print('offerPrice::',offerPrice)
		# input('----------------------------')
		

		n=  n+1
		ws.write(n,0,url)
		ws.write(n,1,productid[0])
		ws.write(n,2,productTitle[0])
		ws.write(n,3,size_type)
		ws.write(n,4,size)
		ws.write(n,5,color)
		ws.write(n,6,stockdteail)
		ws.write(n,7,originalPrice)
		ws.write(n,8,currentprice)
		# ws.write(n,9,offerPrice)
		ws.write(n,10,final_images)
		ws.write(n,11,des)
		wb.save('landsend.xls')